In [1]:
import os
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import OneHotEncoder

In [2]:
# Числовые признаки
num_cols = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

# Категориальные признаки
cat_cols = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

no_cat = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent',
    'Churn'
]

feature_cols = num_cols + cat_cols
col_count = len(feature_cols)
unique_cat = []
target_col = 'Churn'
error_col = 'TotalSpent'

In [3]:
# GET_DUMMIES VERSION

# # train set download
# data = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/train.csv')
# data = data.dropna()

# # похоже в этом столбце некорректный формат данных
# data[error_col] =  pd.to_numeric(data[error_col], errors='coerce')
# data = data.dropna()

# # numeric data settings
# scaler = StandardScaler()
# data[num_cols] = scaler.fit_transform(data[num_cols])

# # categorical data settings
# data[cat_cols] = data[cat_cols].apply(LabelEncoder().fit_transform)
# data = pd.get_dummies(data, columns=cat_cols)

# # test set download
# X_submis = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/test.csv')
# X_submis[error_col] =  pd.to_numeric(X_submis[error_col], errors='coerce')
# median = X_submis[error_col].sum()//len(X_submis[error_col])
# X_submis = X_submis.fillna(median)

# X_submis[cat_cols] = X_submis[cat_cols].apply(LabelEncoder().fit_transform)
# X_submis = pd.get_dummies(X_submis, columns=cat_cols)
# X_submis[num_cols] = scaler.fit_transform(X_submis[num_cols])

# submission = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/submission.csv')

In [4]:
# # DEBAG CELL

# # train set download
# data = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/train.csv')
# data = data.dropna()

In [5]:
# data

In [6]:
# # DEBAG CELL

# data[cat_cols]

In [7]:
def get_unique_cat(df, cat_cols):
    
    global unique_cat
    unique_cat = [] #debag
    
    for col in cat_cols:
        unique_cat += [f'{col}_{item}' for item, cat in enumerate(list(df[col].unique()))]
    
    return unique_cat
# len(list(df[col].unique()))

In [8]:
def df_constructor(main_df, encode_arr, cols):
    global num_cols
    
    df_encoded = pd.DataFrame(encode_arr, columns=cols)
    
    main_df = main_df[no_cat].reset_index(drop=True)
    full_df = main_df.join(df_encoded)
    return full_df

In [9]:
# ONE_HOT_ENCODE + INDEX UPDATE VERSION

# train set download
data = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/train.csv')

# похоже в этом столбце некорректный формат данных
data[error_col] =  pd.to_numeric(data[error_col], errors='coerce')
data = data.dropna()

# numeric data settings
scaler = StandardScaler()
data[num_cols] = scaler.fit_transform(data[num_cols])

# categorical data settings
unique_cat_arr = get_unique_cat(data[cat_cols], cat_cols)
encoder = OneHotEncoder(sparse=False, handle_unknown = 'ignore')
data[cat_cols] = data[cat_cols].apply(LabelEncoder().fit_transform)
data = data.dropna()

encoded_arr = encoder.fit_transform(data[cat_cols])
coded_data = df_constructor(data, encoded_arr, unique_cat_arr)

# test set download
X_submis = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/test.csv')
X_submis[error_col] =  pd.to_numeric(X_submis[error_col], errors='coerce')
median = X_submis[error_col].sum()//len(X_submis[error_col])
X_submis = X_submis.fillna(median)

X_submis[cat_cols] = X_submis[cat_cols].apply(LabelEncoder().fit_transform)
X_submis = pd.get_dummies(X_submis, columns=cat_cols)
X_submis[num_cols] = scaler.fit_transform(X_submis[num_cols])

submission = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/submission.csv')

In [10]:
# DEBAG CELL

coded_data

,ClientPeriod,MonthlySpending,TotalSpent,Churn,Sex_0,Sex_1,IsSeniorCitizen_0,IsSeniorCitizen_1,HasPartner_0,HasPartner_1,...,HasMovieSubscription_2,HasContractPhone_0,HasContractPhone_1,HasContractPhone_2,IsBillingPaperless_0,IsBillingPaperless_1,PaymentMethod_0,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3
0,0.919099,-1.506436,-0.557582,0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1.612060,-1.295997,-0.184763,0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
2,-1.282072,0.362658,-0.976504,1,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,-0.018437,0.475334,0.122800,0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,1.122911,1.666716,1.968909,0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5268,-1.200547,-1.145210,-0.973749,0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5269,0.715287,-0.679594,-0.045372,0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
5270,-1.282072,-0.328310,-0.985693,0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
5271,-0.140724,0.365972,-0.033560,0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [11]:
y = coded_data[target_col]

features_list = list(coded_data)
features_list.remove(target_col)

X = coded_data[features_list]
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, 
                                                    train_size=0.8,
                                                    random_state=42)

X_submis = X_submis.to_numpy()

In [12]:
C_l = [100, 10, 1, 0.1, 0.01, 0.001]

param_grid = {'C': C_l}

search = GridSearchCV(LogisticRegression(max_iter=10000), 
                      param_grid,
                      cv=10,
                      refit=True)

pipe = make_pipeline(
    OneHotEncoder(handle_unknown='ignore'),
    StandardScaler(with_mean=False),
    search
)
pipe.fit(X_train, y_train)

Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                ('standardscaler', StandardScaler(with_mean=False)),
                ('gridsearchcv',
                 GridSearchCV(cv=10,
                              estimator=LogisticRegression(max_iter=10000),
                              param_grid={'C': [100, 10, 1, 0.1, 0.01,
                                                0.001]}))])

In [13]:
preds_train = pipe.predict(X_test)

In [14]:
print("Tuned Hyperparameters :", search.best_params_)
print("Accuracy :",search.best_score_)

Tuned Hyperparameters : {'C': 0.001}
Accuracy : 0.7769016446961083


In [15]:
preds_test = pipe.predict(X_submis)

In [16]:
new_df = pd.DataFrame(preds_test)
print(preds_test.shape)

(1761,)


In [17]:
# submission['#']

In [18]:
submission[target_col] = new_df
os.chdir('/kaggle/working/')
submission.to_csv('submission_.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
